In [1]:
import os
import sys
from pathlib import Path
PATH = os.path.join(os.getcwd(), '..', '..')
sys.path.append(PATH)

import torch
import numpy as np
import matplotlib.pyplot as plt

from fastai.vision import *

from src.model_utils.cyclegan import CycleGAN
from src.model_utils.databunch import ImageTupleList
from src.model_utils.callbacks import CycleGANTrainer
from src.model_utils.loss import CycleGanLoss

%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
data_dir = Path('../../data/test')

In [3]:
data = (ImageTupleList.from_folders(data_dir, 'trainA', 'trainB')
                          .split_none()
                          .label_empty()
                          .transform(get_transforms(), size = 56)
                          .databunch(bs = 1))
cycle_gan = CycleGAN(ch_in = 3, ch_out = 3, 
                     disc_layers = 3, 
                     gen_blocks = 5)

cb = partial(CycleGANTrainer, base_dir = data_dir, name = 'v1', 
        loss_iters = 1, hist_iters = 1, stats_iters = 1, critic_period = 2, warmup_iters = 50
        )
learn = Learner(data, cycle_gan, 
                loss_func = CycleGanLoss(cycle_gan), 
                opt_func = partial(optim.Adam, betas = (0.5, 0.99)),
                callback_fns = [cb])

In [4]:
learn.fit(10)

epoch,train_loss,valid_loss,id_loss,gen_loss,cyc_loss,D_A_loss,D_B_loss,time
0,8.346087,#na#,1.851231,2.789056,3.705801,1.077692,0.721084,00:04
1,7.068188,#na#,1.425332,2.787915,2.854941,1.079274,0.713918,00:05
2,6.644080,#na#,1.309222,2.710155,2.624705,1.079728,0.721432,00:05
3,6.343055,#na#,1.232895,2.611045,2.499115,1.078798,0.726602,00:05
4,6.055334,#na#,1.174819,2.469502,2.411013,1.072669,0.732835,00:06
5,11.486148,#na#,1.163939,7.947424,2.374783,5.005979,3.337289,00:06
6,9.866767,#na#,1.155540,6.364695,2.346530,3.856902,2.622132,00:06
7,8.801770,#na#,1.153007,5.315485,2.333277,3.004384,2.054406,00:06
8,8.075779,#na#,1.123541,4.683922,2.268314,2.465895,1.641428,00:07
9,7.412836,#na#,1.092044,4.120012,2.200780,1.956116,1.318690,00:07
